## Data: https://www.kaggle.com/adarshkumarjha/email-spam-detection-sentiment-analysis

In [2]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import *
from sklearn.metrics import *
import nltk
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import string
from nltk.tokenize import word_tokenize
import math
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from nltk import pos_tag
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [3]:
stop_words=set(stopwords.words('english'))

In [4]:
df=pd.read_csv('emails.csv')

In [5]:
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [6]:
for i in range(len(df)):
    df.text[i]=word_tokenize(df.text[i])

In [7]:
df.head()

,text,spam
0,"[Subject, :, naturally, irresistible, your, co...",1
1,"[Subject, :, the, stock, trading, gunslinger, ...",1
2,"[Subject, :, unbelievable, new, homes, made, e...",1
3,"[Subject, :, 4, color, printing, special, requ...",1
4,"[Subject, :, do, not, have, money, ,, get, sof...",1


In [8]:
filtered_sentence=[]
for i in range(len(df)):
    for word in df.text[i]:
        if word not in stop_words:
            filtered_sentence.append(word)
    df.text[i]=filtered_sentence
    filtered_sentence=[]

In [9]:
df.head()

,text,spam
0,"[Subject, :, naturally, irresistible, corporat...",1
1,"[Subject, :, stock, trading, gunslinger, fanny...",1
2,"[Subject, :, unbelievable, new, homes, made, e...",1
3,"[Subject, :, 4, color, printing, special, requ...",1
4,"[Subject, :, money, ,, get, software, cds, !, ...",1


In [10]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [11]:
word_stems = []
for i in range(len(df)):
    for word in df.text[i]:
        word_stems.append(stemmer.stem(word))
    df.text[i]=word_stems
    word_stems=[]

In [12]:
df.head()

,text,spam
0,"[subject, :, nat, irresist, corp, id, lt, real...",1
1,"[subject, :, stock, trad, gunsl, fanny, merril...",1
2,"[subject, :, unbeliev, new, hom, mad, easy, im...",1
3,"[subject, :, 4, col, print, spec, request, add...",1
4,"[subject, :, money, ,, get, softw, cds, !, sof...",1


In [16]:
import re

In [36]:
all_words=[]
for i in range(len(df)):
    for word in df.text[i]:
        if word not in [':',';',',','.','-','\\','[',']','?','','_']:
            all_words.append(word.strip().lower())
    df.text[i]=all_words
    all_words=[]

In [37]:
df.head()

,text,spam
0,"[subject, nat, irresist, corp, id, lt, real, h...",1
1,"[subject, stock, trad, gunsl, fanny, merril, m...",1
2,"[subject, unbeliev, new, hom, mad, easy, im, w...",1
3,"[subject, 4, col, print, spec, request, addit,...",1
4,"[subject, money, get, softw, cds, !, softw, co...",1


In [41]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [83]:
vectorizer = CountVectorizer(lowercase=False)

In [84]:
df.shape

(5728, 2)

In [85]:
from sklearn.model_selection import train_test_split

In [86]:
X=df.text
y=df.spam

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=14,test_size=0.25)

In [88]:
df.text[0][0:10]

['subject',
 'nat',
 'irresist',
 'corp',
 'id',
 'lt',
 'real',
 'hard',
 'recollect',
 'company']